In [6]:
import time
import json
import ast
import os
import datetime
from rich.progress import track

import io
from collections import defaultdict

import imageio
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
from rich.progress import track
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
%matplotlib inline
pd.set_option('display.max_rows', 500)
import json
import cv2
import os
import multiprocessing as mp    

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dl.utils.io_utils import normalize_image
import imageio
from pathlib import Path

%matplotlib inline
from cv.core.image_quality_server_side import ImageQuality

# from cv.core.image_quality_server_side_halo import ImageQuality
from skimage.color import label2rgb

import albumentations as A
import random
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

In [7]:
directory = '/data2/jupiter/datasets/halo_rgb_stereo_train_v6_1/'
csv_name = 'master_annotations.csv' 
from dl.dataset.datamodes.npz.rgbd import RGBDNPZ
dataloader = RGBDNPZ(directory)
albumentation_transform_path = "/home/alex.li/git/JupiterCVML/kore/configs/data/albumentations/seg_trivialaugment.yml"

In [8]:
df = pd.read_csv(directory + csv_name)
df = df.drop_duplicates('unique_id')
df = df.sample(frac=1, ignore_index=True)

/home/alex.li/miniconda3/envs/cvml/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: DtypeWarning: Columns (2,3,24,82,93,94,95,96,97,99,100,102,103,104,105,106,107,108,109,110,120,153,157,158,163,174,177,180,181,187,189,199,200,202,203,204,205,225,230,231,232,235,237,239,240,251,252,254,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,275,279,280,281,284,285,286,288) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
def plot_several_images(img_list, titles=None):
    cols = 4
    rows = (len(img_list) + 3) // cols
    fig, ax = plt.subplots(rows, cols, squeeze=False, figsize = (32, rows * 8))
    for i, img in enumerate(img_list):
        ax[i // cols][i % cols].imshow(img)
        if titles is not None:
            ax[i // cols][i % cols].set_title(titles[i])

In [22]:
sample_images = []
sample_labels = []

for i in range(100):
    artifacts= dataloader.get_artifacts(df.iloc[i])
    im = normalize_image(artifacts['image'], hdr_mode=df.iloc[i]['hdr_mode'])
    sample_images.append(im)
    sample_labels.append(artifacts['label'])

In [69]:
import ruamel.yaml
yaml = ruamel.yaml.YAML(typ="safe")

with open(albumentation_transform_path) as f:
    aug = yaml.load(f)

if isinstance(aug, dict):
    aug = A.from_dict(aug)

In [70]:
cv2.INTER_NEAREST

0

In [71]:
cv2.BORDER_CONSTANT

0

In [72]:
# aug = A.ReplayCompose([
#     A.FromFloat(dtype='uint8'),
#     A.FancyPCA(
#     alpha=4,
#     p=1.0,
# ),
#     A.ToFloat(),
# ])
# aug = A.Solarize(
#     p=1.0,
#     threshold=.3
# )

aug_images = []
aug_titles = []
for i in range(50):
    out = aug(image=sample_images[i], mask=sample_labels[i])
    aug_images.append(sample_images[i])
    aug_titles.append('orig')
    aug_images.append(out['image'])
    if 'replay' in out:
        try:
            int_transforms = [x for x in out['replay']['transforms'][0]['transforms'][0]['transforms'][1]['transforms'] if x['applied']]
            float_transforms = [x  for x in out['replay']['transforms'][0]['transforms'][1]['transforms'] if x['applied']]
            applied = (int_transforms + float_transforms)[0]
        except KeyError:
            applied = out['replay']['transforms'][1]
        print(applied['__class_fullname__'] + str(applied['params']))
        aug_titles.append(applied['__class_fullname__'])
    else:
        aug_titles.append("aug")

    if not np.all(out['mask'] == sample_labels[i]):
        aug_titles.append('orig mask')
        aug_images.append(sample_labels[i])
        aug_titles.append('aug mask')
        aug_images.append(out['mask'])  

plot_several_images(aug_images, aug_titles)